In [ ]:
def significant_points(data):
    FoldChange = defaultdict(list)
    for keys, items in data.items():
        for key, item in items.items():
            FoldChange[key].append(item)
    null_mean = 0.0
    p_values = {}

    for vgene, fold_changes in FoldChange.items():
        if np.var(fold_changes) == 0:
            p_value == 1.0
        else:
            t_test, p_value = stats.ttest_1samp(fold_changes, null_mean)
        p_values[vgene] = p_value

    volcano_plot = pd.DataFrame()
    FC_dataframe = pd.DataFrame.from_dict(FoldChange, orient='index')
    volcano_plot['mean'] = FC_dataframe.mean(axis=1)
    volcano_plot['p-value'] = p_values.values()
    volcano_plot['std'] = FC_dataframe.std(axis=1)
    volcano_plot['-log10(p-value)'] = -np.log10(volcano_plot['p-value'])
    volcano_plot.dropna(inplace=True)
    significance_threshold = 0.01
    significant_points = volcano_plot[volcano_plot['p-value'] < significance_threshold]
    return significant_points

sjo = significant_points(FoldChange_sjogren)
heal = significant_points(FoldChange_healthy)

comm = set(sjo.index).intersection(set(heal.index))

sjo = sjo[sjo.index.isin(comm)]
heal = heal[heal.index.isin(comm)]

plt.figure(figsize=(10, 6))
plt.scatter(sjo['mean'], sjo['-log10(p-value)'].apply(lambda x: (x if x > 0 else 0)), c='r', alpha=0.5, label='sjogren')
plt.scatter(heal['mean'], heal['-log10(p-value)'].apply(lambda x: (x if x > 0 else 0)), c='b', alpha=0.5, label='healthy')

plt.xlabel('log2(Fold Change)')
plt.ylabel('-log10(P-Value)')
plt.title('Fold Change for V genes')

for i, txt in enumerate(sjo.index):
    plt.annotate(txt.split('V')[1], (sjo['mean'][txt], sjo['-log10(p-value)'][txt]))

plt.plot([sjo['mean'], heal['mean']],
                 [sjo['-log10(p-value)'], heal['-log10(p-value)']],
                 color='gray', linestyle='--')

plt.errorbar(sjo['mean'], sjo['-log10(p-value)'],
                     xerr=sjo['std'], fmt='o', color='r', markersize=1, alpha=0.3)
plt.errorbar(heal['mean'], heal['-log10(p-value)'],
                     xerr=heal['std'], fmt='o', color='b', markersize=1, alpha = 0.3)

plt.legend()
plt.savefig('../Data/volcano_plot/comm_over_p_0.05_error.png', dpi=500)
plt.show()